In [1]:
import numpy as np
from keras.layers import Input, Dense, Dropout
from keras.models import Sequential, Model, load_model
from keras import losses
from keras.datasets import mnist
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.externals import joblib
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame
import xlsxwriter
import time

IBIS_Data = pd.ExcelFile("SA_and_CT_AALandfROI_08272019.xlsx") 
IBIS_CONTE_Data = pd.ExcelFile("IBIS 1 and 2 Year Destrieux.xlsx")
scalerCT = MinMaxScaler()
scalerSA = MinMaxScaler()

Using TensorFlow backend.
C:\Users\mattbeze\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [4]:
ct = IBIS_CONTE_Data.sheet_names[0]
sa = IBIS_CONTE_Data.sheet_names[1]

ct_data = IBIS_CONTE_Data.parse(ct)
sa_data = IBIS_CONTE_Data.parse(sa)
ct_data_features = ct_data.loc[:, ct_data.columns] 
ct_data_features = ct_data_features.drop(['Case','Visit','ROI42','ROI117'], axis=1)  
sa_data_features = sa_data.loc[:, sa_data.columns] 
sa_data_features = sa_data_features.drop(['Case','Visit','ROI42','ROI117'], axis=1)  

scaled_ct = scalerCT.fit_transform(ct_data_features)
scaled_sa = scalerSA.fit_transform(sa_data_features)

In [5]:
#Saving the scalers
scalerCT_filename = "IBIS_scaledCT_1_2_year.save"
joblib.dump(scalerCT, scalerCT_filename)
scalerSA_filename = "IBIS_scaledSA_1_2_year.save"
joblib.dump(scalerSA, scalerSA_filename)

['IBIS_scaledSA_1_2_year.save']

In [10]:
#Now transform the Gilmore data set using the scalers based on the IBIS data set
Gilmore_Data = pd.ExcelFile("Gilmore CT1y2y SA1y2y.xlsx") 
Gilmore_CT1y = Gilmore_Data.sheet_names[0]
Gilmore_SA1y = Gilmore_Data.sheet_names[1]
Gilmore_CT2y = Gilmore_Data.sheet_names[2]
Gilmore_SA2y = Gilmore_Data.sheet_names[3]

Gilmore_CT1y_data = Gilmore_Data.parse(Gilmore_CT1y)
Gilmore_SA1y_data = Gilmore_Data.parse(Gilmore_SA1y)
Gilmore_CT2y_data = Gilmore_Data.parse(Gilmore_CT2y)
Gilmore_SA2y_data = Gilmore_Data.parse(Gilmore_SA2y)

Gilmore_CT1y_data_features = Gilmore_CT1y_data.loc[:, Gilmore_CT1y_data.columns] 
Gilmore_CT1y_data_features = Gilmore_CT1y_data_features.drop(['ROI'], axis=1)
Gilmore_SA1y_data_features = Gilmore_SA1y_data.loc[:, Gilmore_SA1y_data.columns] 
Gilmore_SA1y_data_features = Gilmore_SA1y_data_features.drop(['ROI'], axis=1) 
Gilmore_CT2y_data_features = Gilmore_CT2y_data.loc[:, Gilmore_CT2y_data.columns] 
Gilmore_CT2y_data_features = Gilmore_CT2y_data_features.drop(['ROI'], axis=1) 
Gilmore_SA2y_data_features = Gilmore_SA2y_data.loc[:, Gilmore_SA2y_data.columns] 
Gilmore_SA2y_data_features = Gilmore_SA2y_data_features.drop(['ROI'], axis=1) 

scaled_Gilmore_CT1y = scalerCT.transform(Gilmore_CT1y_data_features)
scaled_Gilmore_SA1y = scalerSA.transform(Gilmore_SA1y_data_features)
scaled_Gilmore_CT2y = scalerCT.transform(Gilmore_CT2y_data_features)
scaled_Gilmore_SA2y = scalerSA.transform(Gilmore_SA2y_data_features)

In [12]:
#Saving the scaled Gilmore data
df0 = pd.DataFrame(scaled_Gilmore_CT1y)
df1 = pd.DataFrame(scaled_Gilmore_SA1y)
df2 = pd.DataFrame(scaled_Gilmore_CT2y)
df3 = pd.DataFrame(scaled_Gilmore_SA2y)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Scaled_Gilmore_Data_CT_SA_1_2.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df0.to_excel(writer, sheet_name='Gilmore CONTE CT 1yr')
df1.to_excel(writer, sheet_name='Gilmore CONTE SA 1yr')
df2.to_excel(writer, sheet_name='Gilmore CONTE CT 2yr')
df3.to_excel(writer, sheet_name='Gilmore CONTE SA 2yr')

# Close the Pandas Excel writer and output the Excel file.
writer.save()